
# Query Based Summarization Solution Playbook

## Introduction

Knowledge bases in enterprises are very common in the industry today and can have extensive number of documents in different categories. Retrieving relevant content based on a user query is a challenging task.  Given a query we were able to retrieve information accurately at the document level using methods such as Page Rank developed and made highly accurate especially by Google,  after this point the user has to delve into the document and search for the relevant information.  With recent advances in Foundation Transformer Models such as the one developed by Open AI the challenge is alleviated by using “Semantic Search” methods by using encoding information such as “Embeddings” to find the relevant information and then to summarize the content to present to the user in a concise and succinct manner.

This Playbook will introduce the releavant use cases and the end-to-end archtitecture for this use case. We wuill take you through the step-by-step process of using various Azure Cognitive Services and more specifically Azure OpenAI's GPT-3 models to perform the downstream task of summarization.

This playbook aims to demonstrate how to link various Azure Cognitive Services together to improve results for common downstream tasks in a large enterprise enviorment. The methods and architecture are extremely customizable to your summarization use case and can be applied to many different datasets.


## Use Cases

This playbook links together two main use cases: **Document Search** and **Document Summarization**. By using different Azure Cognitive Services specialized for each task, the end result is focused, succinct, and thoroughly vetted.

### Document Search

Document Search is driven by an initial user query. The user provides their query to the search service, enabling users to find the top most relevant document from their knowledge base. Document Search leverages the state of the art search technology using indexers. Within large enteprises there are various documents in different formats that are congregated in a large knowledge base. Having the ability to quickly, effectively, and accurately find relevant documents is essential to this end-to-end scenario.

### Document Summarization

Document Summarization is performed by providing input quality text into the summarizaton engine. The key to successful document summarization is retrieving the most important points within a large document to feed into summarization, as the quality and focus of a summary is only as good as the input data. To ensure the most succinct and focused summary is returned, we utitlize Embeddings to search within a document for the most salient pieces. Document Summarization is essential in large enteprises to condense large documents into human consumable information that frees up humans to do more specialized and techincal work rather than focus on common tasks such as summmarization.

## Architecture

An E2E architecture found in Images/EndToEndArchitecture.png captures the use case that we talked about the previous sections. The user passes their query to a knowledge base to filter the number of documents top most relevant docouments, narrows the scope of documents to tens from a couple of thousands. This cna be done using Azure Cognitive Search by creating a custom index for your knowledge base. Then once a document is selected, the document needs to be segmented and then each segment can be embedded using GPT-3 embedding models. This allows each text chunk to have its semantic meaning captured through an embedding. Then the user can send another query to search within the document and "zone in" on a particular text segment or segements based using similarities between the query and embeddings for each text chunk. Once a section is "zoned-in", the user can extract the relevant text and pass it to the GPT-3 Completion endpoint for summarization. 

## Example Walk-Through

This remainder of this notebook will dive deep into each component of the architecture and demonstrate how to link them together for a successful document summarization use case. 

Our guiding query that we will use for this playbook is **"Provide details on the Clinton democratic nomination"**

## Dataset

For this walkthrough, we will be using the **CNN/Daily Mail** dataset. This is a common dataset used for text summarization and question answering tasks. Human generated abstractive summary bullets were generated from news stories in CNN and Daily Mail websites. 

In all, the corpus has 286,817 training pairs, 13,368 validation pairs and 11,487 test pairs, as defined by their scripts. The source documents in the training set have 766 words spanning 29.74 sentences on an average while the summaries consist of 53 words and 3.72 sentences. We will use a subset of the corpus for this example that can be found in the /data folder. 

The relevant schema for our work today consists of:

+ id: a string containing the heximal formatted SHA1 hash of the URL where the story was retrieved from
+ article: a string containing the body of the news article
+ highlights: a string containing the highlight of the article as written by the article author


In [ ]:
%pip install openai
%pip install plotly
%pip install python-dotenv
%pip install azure-storage-blob azure-identity

In [ ]:
import pandas as pd

dataset = pd.read_json("cnn_dataset.json")

dataset.info()
dataset.head()

## Knowledge Base: __Azure Blob Storage__

Being able to store and access unstructured data at scale is imperative for enterprises. Azure Blob Storage provides storage capabilities with optimized costs with tieired storage for long-term data, and flexibiluty for high perofrmancing computer and ML workloads. The comprehensive data management Azure Blob Storage provides coupled with it's ease of use is why we chose to upload our knowledge base to Azure Blob Storage for this scenario.

Around 11.2K articles and their summaries are stored in the Azure cloud for quick aceess. To do this we created a blob services with the dataset uploaded as a jsonl. This blob service is contained wihtin a storage account this will link to other services further in this example.

For more information please reference: https://learn.microsoft.com/en-us/azure/storage/blobs/storage-blobs-introduction

In [ ]:
import os
from dotenv import load_dotenv
AZURE_STORAGE_CONNECTION_STRING=os.getenv("AZURE_STORAGE_CONNECTION_STRING") # SET YOUR API KEY in a config.env file in the same folder as this notebook
print(AZURE_STORAGE_CONNECTION_STRING)


## Enterprise Search: __Azure Cognitive Search__

Azure Cognitive Search is a cloud search service that provides developers infrastructure, APIs, and tools for building a rich search experience. This service creates a search engine for full text search over a search index containing user-owned content. Azure Cognitive Search uses semantic search that brings relevance and language understanding to search results.

In this example, we will create a search service within the Azure portal with a customized index that enables us to search the CNN/Daily Mail knowledge base stored in Azure Blob storage.

In the image below, we have created an index to search against the 11.2K document stored in the Azure Blob Storage. We use the id, article, and highlights field to be retrievable during the search.

[insert image here in MD images\AzureCogSearchIndex]

Now that we have our our storage and search services set up through the Azure portal, we can dive into our use case. 

The user is now able to query against the knowledge base. As a reminder from above, our **guiding initial query is "Provide details on the Clinton Democratic nomination"**

Lets place that query or a paragraphse of the query into the Azure Cognitive Search service, and see what the top results are.

[insert image for AzureCogSearchResults]

Lets take a deeper dive into the results below.

In [17]:
import os
import re
import sys
import pandas as pd
import requests
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv

index_name = "azureblob-index"
# Get the service endpoint and API key from the environment
SEARCH_ENDPOINT = os.getenv("SEARCH_ENDPOINT")
print(SEARCH_ENDPOINT)
SEARCH_API_KEY = os.getenv("SEARCH_API_KEY")
print (SEARCH_API_KEY)

# Create a client
credential = AzureKeyCredential(SEARCH_API_KEY)
client = SearchClient(endpoint=SEARCH_ENDPOINT,
                      index_name=index_name,
                      credential=credential)
results = client.search(search_text="clinton democratic nomination")
search_df = pd.DataFrame(results)
search_df.head(3)




  
  

https://achandaisearchx1.search.windows.net/
GkLjS91JavBs4AeXfjDOJ1gqBKV453uYTntVukGp92AzSeAs6fCC


,highlights,article,AzureSearch_DocumentKey,id,@search.score,@search.highlights
0,Hillary Clinton developed a close relationship...,New York (CNN)Wall Street is more than ready f...,aHR0cHM6Ly9hY2hhbmRhaXN0b3JleDEuYmxvYi5jb3JlLn...,22e1394839f0ec18ee80df0cbf3f2d5ce22bff2b,33.520344,None
1,Laurene Jobs praised former First Lady Hillary...,Apple founder Steve Jobs' widow Laurene has to...,aHR0cHM6Ly9hY2hhbmRhaXN0b3JleDEuYmxvYi5jb3JlLn...,faf5a752ae42c7e325f93cebef744e02338a9a0c,31.193993,None
2,"'Republicans seem to be talking only about me,...",Hillary Rodham Clinton took aim at Republicans...,aHR0cHM6Ly9hY2hhbmRhaXN0b3JleDEuYmxvYi5jb3JlLn...,f29253fb79c3560d9f24bd1531cf4e0b20e2214f,26.876034,None


Here we have selected the top 3 results from our Azure Cognitive Search app. In the results we can see the search score Azure Cognitive Search uses to assess semantic similarity between the query and the knowledge base. 

At this point in the use case, the user can either select the top article or investigate which of the top results is providing the most relevant information for their use case. By putting ourselves in the customer shoes and reflecting back on our guiding initial query of **clinton democratic nomination**, lets move forward with the first article. 

The first article highlights Clinton's campaign kick off in the Iowa caucused and focuses on her seeking the democratic nomation. The top search result matched with our initial query!

## Document Zone: __Azure OpenAI Embedding API__

Now that we have selected a narrowed on a single document from our knowledge base of 11.2K documents - we can dive deeper into the single document to refine our initial query to a more specific section or "zone" of the article.

To do this, we will utilize the Azure Open AI Embeddings API. 

### Embeddings Overview

An embedding is a special format of data representation that can be
easily utilized by machine learning models and algorithms. The embedding
is an information dense representation of the semantic meaning of a
piece of text. Each embedding is a vector of floating-point numbers,
such that the distance between two embeddings in the vector space is
correlated with semantic similarity between two inputs in the original
format. For example, if two texts are similar, then their vector
representations should also be similar.

Different Azure OpenAI embedding models are specifically created to be
good at a particular task. Similarity embeddings are good at capturing
semantic similarity between two or more pieces of text. Text search
embeddings help measure long documents are relevant to a short query.
Code search embeddings are useful for embedding code snippets and
embedding nature language search queries.

Embeddings make it easier to do machine learning on large inputs
representing words by capturing the semantic similarities in a vector
space. Therefore, we can use embeddings to if two text chunks are
semantically related or similar, and inherently provide a score to
assess similarity.

### Cosine Similarity

A previously used approach to match similar documents was based on
counting maximum number of common words between documents. This is
flawed since as the document size increases, the overlap of common words
increases even if the topics differ. Therefore cosine similarity is a
better approach by using the Euclidean distance.

Mathematically, cosine similarity measures the cosine of the angle
between two vectors projected in a multi-dimensional space. This is
beneficial because if two documents are far apart by Euclidean distance
because of size, they could still have a smaller angle between them and
therefore higher cosine similarity.

The Azure OpenAI embeddings rely on cosine similarity to compute
similarity between documents and a query.

### Text Segmentation or "Chunking"

The document that we selected span a few pages. In order to produce a meaningful and a focused summary we must first chunk or segment the document. This is essential for long document summarization for two main reasons:

+ Going around the token limitation inherit in a transformer based model - due to the token limitation we cannot pass the entire document into a model 
+ Creating a mapping from topic to relevant chunks of text - for long documents topics can vary drastically and to produce a meaningful summary, most of the time you want to "zone" on a single area. This may be a page or just a section with the information you want to dive into. 


By chunking the document into logical segments, we can utilize the power of the Azure OpenAI Embeddings following these steps:

1. Chunk the document into logical segments that fit within the token limitation
2. Create an embedding vector for each chunk that will capture the semantic meaning and overall topic of that chunk
3. Upon receiving a query for the specific document, embed the query in the same vector space as the context chunks from Step 2.
4. Find the most relevant context chunks through cosine similarity and use it for your desired downstream tasks. 

Let's walk through this together.

In [ ]:
# This is the original text from the top search result from Azure Cognitive Search
document=search_df._get_value(0,'article')
print(document)

In [19]:
import openai
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # SET YOUR API KEY in a config.env file in the same folder as this notebook
OPENAI_ENDPOINT = os.getenv("OPENAI_ENDPOINT") # SET A LINK TO YOUR RESOURCE ENDPOINT in a config.env file in the same folder as this notebook
openai.api_type = "azure"
openai.api_key = OPENAI_API_KEY
openai.api_base = OPENAI_ENDPOINT
openai.api_version = "2022-06-01-preview"

TEXT_SEARCH_DOC_EMBEDDING_ENGINE = 'text-search-curie-doc-001'
TEXT_SEARCH_QUERY_EMBEDDING_ENGINE = 'text-search-curie-query-001'
TEXT_DAVINCI_001 = "davinci-instruct"

In [20]:
#Defining helper functions

#Splits text after sentences ending in a period. Combines n sentences per chunk.
def splitter(n, s):
    pieces = s.split(". ")
    list_out = [" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n)]
    return list_out

# Perform light data cleaning (removing redudant whitespace and cleaning up punctuation)
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s


In [21]:
# (STEP 1) Chunking the document by creating a chunk every 10th sentence and creating a pandas DF
document_chunks = splitter(10, normalize_text(document))
df = pd.DataFrame(document_chunks, columns = ["chunks"])
df.head()

,chunks
0,New York (CNN)Wall Street is more than ready f...
1,"Robert Wolf, the former CEO of UBS Americas an..."
2,"In January, she took to social media to defend..."
3,"Martin O'Malley, former Virginia Sen Jim Webb ..."
4,Democratic strategist Chris Lehane downplayed ...


In [22]:
#(STEP 2): Create an embedding vector for each chunk that will capture the semantic meaning and overall topic of that chunk
df['curie_search'] = df["chunks"].apply(lambda x : get_embedding(x, engine = TEXT_SEARCH_DOC_EMBEDDING_ENGINE))

In [23]:
df

,chunks,curie_search
0,New York (CNN)Wall Street is more than ready f...,"[-0.002445351565256715, 0.0039451043121516705,..."
1,"Robert Wolf, the former CEO of UBS Americas an...","[-0.0030941024888306856, -0.005450507160276174..."
2,"In January, she took to social media to defend...","[-0.008894749917089939, 0.008842174895107746, ..."
3,"Martin O'Malley, former Virginia Sen Jim Webb ...","[-0.0007511721341870725, -0.01596105843782425,..."
4,Democratic strategist Chris Lehane downplayed ...,"[-0.010210839100182056, 0.003850820939987898, ..."


Now that we have a dataframe with the document chunks and their corresponding embedding vectors. We can "zone in" more specifically on a certain area of the document by narrowing our query. 

Lets generate a document specific query that will help us isolate the correct context chunks that we can pass down into further downstream tasks.

The document specific query that we will use for Step 3 is **what is the trouble so far in the clinton campaign?**

In the code section below, we will pass the document specific query to a function that embeds the query to the same vector space as the document chunks. Calculate the cosine similarity between each context chunk and the query to find the most relevant context chunk based on the input query.

In [24]:
# (STEP 3) upon receiving a query for the specific document, embed the query in the same vector space as the context chunks
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine=TEXT_SEARCH_QUERY_EMBEDDING_ENGINE
    )
    df["similarities"] = df.curie_search.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

document_specific_query = "trouble so far in clinton campaign"
res = search_docs(df, document_specific_query, top_n=2)

,chunks,curie_search,similarities
4,Democratic strategist Chris Lehane downplayed ...,"[-0.010210839100182056, 0.003850820939987898, ...",0.354676
2,"In January, she took to social media to defend...","[-0.008894749917089939, 0.008842174895107746, ...",0.353955


Let's take a deep dive into the results from step 3. They both have very similar cosine simialrity to the input query. Both results talk about current trouble the Clinton campaign is having by staging "ordinary voter" and creating fake events in Iowa where the campaign was kick off. 

The resulting context chunks are relevant to our query and can be used for further downstream tasks. At this point, the user can decide if they want to just take the top result to the downstream task or combine a few context chunks. (Step 4)

For this example, let's use both of the chunks as they are both relevant to the Clinton nomination and issues so far during the campaign for our downstream task of summarization.

In [26]:
result_1 = res["chunks"][4]
print(result_1)

Democratic strategist Chris Lehane downplayed the notion of Clinton-Warren tension in the Democratic Party, predicting that with Clinton as an announced candidate, "she'll offer a pretty compelling rationale" for her campaign Lehane, who worked in Bill Clinton's administration, said he envisioned an economic message for Clinton anchored in her biography: "I grew up in the Midwest in a middle class family, I understand the challenges that they face, we need to make sure that America gives people a fair shot." Despite Clinton's embrace of more populist rhetoric, finance and business leaders aren't too concerned that she will back policies that are anathema to them They expect that she will be able to articulate a broad economic goals aimed at the middle class rather than one that rails against bailouts and financial excess, particularly as the country gets more distance from the last financial crisis Kathy Wylde, president of the Partnership for New York City, a prominent business coalit

In [27]:
result_2 = res["chunks"][2]
print(result_2)

In January, she took to social media to defend the 2010 Dodd-Frank Wall Street reform law, which contains provisions that the industry has tried to roll back "Attacking financial reform is risky and wrong," she wrote on Twitter Bill Daley, a longtime Clinton ally and former Obama chief of staff, said Clinton has to reintroduce herself to the party If she defends policies viewed as having contributed to the financial crisis, Daley said, "that's a problem." He continued: "My guess is she'll have enough policy positions that says she's not in the tank with them." Clinton's early gestures have not satisfied some activists, who point to Massachusetts Democratic Sen Elizabeth Warren as their candidate of choice Several liberal groups have even launched a formal draft campaign to elevate the senator and highlight her progressive views Former Republican New Hampshire Sen Judd Gregg, who served as head of the Securities Industry and Financial Markets Association, said part of Clinton's challeng

## Document Summarization: __Azure OpenAI Completion API__

Now that we have extracted the relevant context chunks using Azure OpenAI Embedding API, we can use these chunks to provide meaningful context for our prompt design. We will be using the **Completion endpoint** for summarization. 

Together, let's design a prompt used for summarizing our extracted texts.

### Prompt Design Refresher

GPT-3 models can perform many tasks. Therefore you must be explicit in describing what you want. 

The models try to guess what you want from the prompt. If you send the words "Give me a list of cat breeds," the model wouldn't automatically assume that you're asking for a list of cat breeds. You could just as easily be asking the model to continue a conversation where the first words are "Give me a list of cat breeds" and the next ones are "and I'll tell you which ones I like." If the model only assumed that you wanted a list of cats, it wouldn't be as good at content creation, classification, or other tasks.

There are three basic guidelines to creating prompts:

**Show and tell**. Make it clear what you want either through instructions, examples, or a combination of the two. If you want the model to rank a list of items in alphabetical order or to classify a paragraph by sentiment, show it that's what you want.

**Provide quality data**. If you're trying to build a classifier or get the model to follow a pattern, make sure that there are enough examples. Be sure to proofread your examples — the model is usually smart enough to see through basic spelling mistakes and give you a response, but it also might assume this is intentional and it can affect the response.

**Check your settings.** The temperature and top_p settings control how deterministic the model is in generating a response. If you're asking it for a response where there's only one right answer, then you'd want to set these lower. If you're looking for more diverse responses, then you might want to set them higher. The number one mistake people use with these settings is assuming that they're "cleverness" or "creativity" controls.

### Few-Shot Approach

The goal of this is to teach the GPT-3 model to learn a conversation style input. We use the “Completion” create OpenAI API and generate a prompt that would best provide us a summary of the conversation. It is important to generate prompts carefully to extract relevant information. To extract general summaries from customer-agent chats, we will be using the following format:

1. Prefix: What do you want it to do
2. Context primer : Describe what the context is
3. Context: # Essentially the information needed to answer the question. In the case of summary, the prose that needs to be summarized. 
4. Suffix: Describe what form the answer should take. Whether it is an answer, a completion, a summary, etc

In [28]:
'''
Designing a prompt that will show and tell GPT-3 how to proceed. 
+ Providing an instruction to summarize the text about the general topic (prefix)
+ Providing quality data for the chunks to summarize and specifically mentioning they are the text provided (context + context primer)
+ Providing a space for GPT-3 to fill in the summary to follow the format (suffix)
'''
prompt_i = 'Summarize the content about the Clinton campaign given the text provided.\n\Text:\n'+" ".join([normalize_text(result_1)])+ '\n\nText:\n'+ " ".join([normalize_text(result_2)])+'\n\nSummary:\n'

prompt_i

'Summarize the content about the Clinton campaign given the text provided.\n\\Text:\nDemocratic strategist Chris Lehane downplayed the notion of Clinton-Warren tension in the Democratic Party, predicting that with Clinton as an announced candidate, "she\'ll offer a pretty compelling rationale" for her campaign Lehane, who worked in Bill Clinton\'s administration, said he envisioned an economic message for Clinton anchored in her biography: "I grew up in the Midwest in a middle class family, I understand the challenges that they face, we need to make sure that America gives people a fair shot." Despite Clinton\'s embrace of more populist rhetoric, finance and business leaders aren\'t too concerned that she will back policies that are anathema to them They expect that she will be able to articulate a broad economic goals aimed at the middle class rather than one that rails against bailouts and financial excess, particularly as the country gets more distance from the last financial crisis

In [29]:
# Using a temperature a low temperature to limit the creativity in the response. 
response = openai.Completion.create(
        engine= TEXT_DAVINCI_001,
        prompt = prompt_i,
        temperature = 0.4,
        max_tokens = 500,
        top_p = 1.0,
        frequency_penalty=0.5,
        presence_penalty = 0.5,
        best_of = 1
    )

print(response.choices[0].text)


The article discusses how some in the Democratic Party are concerned that Hillary Clinton may be too centrist and pro-business for the party's base, and that she may have trouble winning the nomination if she is challenged from the left. It also mentions that Clinton has defended Dodd-Frank Wall Street reform and has taken to social media to defend it.


As a result, we have a succinct, clear, and impactful summary generated by the Azure OpenAI Completion API.

## Review

To summarize, we started by transferring a large knowledge base of data to **Azure Blob Storage**. We also deployed a **Azure Cognitive Search** service by creating a custom indexer for our data. From there, the user provided an initial query in the Azure Cognitive Search app that searched against the Blob Storage. The **enterprise search** results were listed as an output based on Azure Cognitive Search's similarity metric. From there, we **isolated the top search result** and extracated the text. At this point we are now focusing on one document relevant to the initial query. From here, we **chunked** our document to break up the long document into individual chunks. Then we utilized **Azure OpenAI Embedding API** to embed each context vector and asssign a representation semantic meaning to each chunk. Then we created a document specific query that will enable us to **"zone in"** on a specific section of the document that we want to use for our downstream task. After passing the document specific query, we used Embeddings to find the most **relevant context chunks** to the query. From there we extracted the relevant context chunks, and designed a **few-shot prompt** for the **Azure OpenAI Completion API** for the downstream task of summarization. 

From starting from a 11.2K document knowledge base to narrowing onto a specific document, and then finding the relevant inforamtion within the document to generate the best summary - linking Azure Cognitive services is an excellent approach.
